BGD、SGD、MBGD：三者的区别取决于我们用多少数据来计算目标函数的梯度。

#### BGD（Batch Gradient Descent）
采用整个训练集的数据来计算cost function对参数的梯度。
$$\theta = \theta - \eta \cdot \bigtriangledown _{\theta}J(\theta)$$

缺点：
1. 由于在一次更新中就对整个数据集计算梯度，所以计算起来非常慢，遇到很大量的数据集也会非常棘手，而且不能投入新数据实时更新模型。
2. 对于凸函数可以收敛到全局极小值，对于非凸函数可以收敛到局部极小值。

______
#### SGD（Stochastic Gradient Descent）
每次更新时对每个样本进行梯度更新，对于很大的数据集来说，可能会有相似的样本，这样BGD在计算梯度时会出现冗余，而SGD一次只进行一次更新，就没有冗余，而且比较快，并且可新增样本。
$$\theta = \theta - \eta \cdot \bigtriangledown _{\theta}J(\theta;x^{(i)};y^{(i)})$$

缺点：
1. SGD的噪音较BGD要多，使得SGD并不是每次迭代都向着整体最优方向，所以虽然训练速度快，但是准确度下降，并不是全局最优。虽然具有一定的随机性，但是从期望上来看，它是等于正确的导数的。

_________
#### MBGD（Mini-Batch Gradient Descent）
每一次利用一小批样本，即$n$个样本进行计算，这样它可以降低参数更新时的方差，收敛更稳定，另一方面可以充分地利用深度学习库中高度优化的矩阵操作来进行更有效的梯度计算。
$$\theta = \theta - \eta \cdot \bigtriangledown _{\theta}J(\theta;x^{(i)};y^{(i)})$$

缺点：
1. MBGD不能保证很好的收敛性，learning rate如果选择的太小，收敛速度会很慢；如果太大，loss function就会在极小值处不停地震荡甚至偏离。
2. 对于非凸函数，还要避免陷于局部极小值处，或者鞍点处，因为鞍点周围的error是一样的，所有维度的梯度都接近于0，SGD很容易困在这里。
3. 在鞍点或者局部最小点处，如果是BGD，则优化会停止不动；如果是MBGD或SGD，每次找到的梯度都是不同的，就会发生震荡，来回跳动。

#### Momentum
$$v_t=\gamma v_{t-1}+\eta \bigtriangledown_{\theta}J(\theta)$$
$$\theta=\theta - v_t$$
使得梯度方向不变的维度上速度变快，梯度方向有所改变的维度上的更新速度变慢，这样就可以加快收敛并减小震荡。$\gamma$为超参数，一般取$0.9$左右。

缺点：此时相当于小球从山上滚下来时是在盲目地沿着坡滚，如果它能具备一些先知，就知道需要减速了的话，适应性会更好。

#### NAG（Nesterov Accelerated Gradient）
用$\theta - \gamma v_{t-1}$来近似当作参数下一步会变成的值。即在计算梯度时，不是在当前的位置，而是未来的位置上。
$$v_t=\gamma v_{t-1}+\eta \bigtriangledown_{\theta}J(\theta - \gamma v_{t-1})$$
$$\theta=\theta - v_t$$

1. 此时，在更新梯度时可顺应loss function的梯度来调整速度，并且对SGD进行加速。
2. 还希望可根据参数的重要性而对不同的参数进行不同程度的更新。

#### Adagrad（Adaptive gradient algorithm）
对低频的参数做较大的更新，对高频的做较小的更新。因此，对于稀疏的数据它的表现很好，很好的提高了SGD的鲁棒性。
$$\theta_{t+1,i}=\theta_{t,i}-\frac{\eta}{\sqrt{G_{t,ii}+\epsilon}}\cdot \bigtriangledown _{\theta}J(\theta _i)$$
其中，$G_{t,ii}$是对角矩阵，$ii$元素是$t$时刻参数$\theta _i$的梯度平方和。

优点：减少了学习率的手动调节。

缺点：分母会不断积累，这样学习率就会收敛并最终会变得非常小。

#### Adadelta
$$\Delta \theta _t=-\frac{\eta}{\sqrt{E[g^2]_t+\epsilon}}g_t=-\frac{\eta}{RMS[g]_t}g_t$$
$$E[g^2]_t=\gamma E[g^2]_{t-1}+(1-\gamma)g^2_t$$
和Adagrad相比，分母的$G$换成了过去的梯度平方的衰减平均值。

将$\eta$换成$RMS[\Delta \theta]$，此时都不需要提前设定学习率了：
$$\Delta \theta _t=-\frac{RMS[\Delta \theta]_{t-1}}{RMS[g]_t}g_t$$
$$\theta _{t+1}=\theta _t +\Delta \theta _t$$
其中，$g_t=\bigtriangledown _{\theta}J(\theta)$

#### RMSprop
RMSprop和Adadelta都是为了解决Adagrad学习率急剧下降问题的。
$$E[g^2]_t=0.9 E[g^2]_{t-1}+0.1g^2_t$$
$$\theta _{t+1}=\theta _t - \frac{\eta}{\sqrt{E[g^2]_t}+\epsilon}g_t$$
建议$\eta$为$0.001$。

#### Adam（Adaptive Moment Estimation）
$$m_t=\beta_1 m_{t-1}+(1-\beta _1)g_t$$
$$v_t=\beta_2 v_{t-1}+(1-\beta_2)g^2_t$$
如果$m_t$和$v_t$被初始化为0向量，它们就会向0偏置，所以需做偏差校正。
$$\hat{m_t}=\frac{m_t}{1-\beta^t_1}$$
$$\hat{v_t}=\frac{v_t}{1-\beta^t_2}$$
梯度更新规则为：
$$\theta _{t+1}=\theta _t - \frac{\eta}{\sqrt{\hat{v_t}}+\epsilon}\hat{m_t}$$
建议取值：$\beta_1=0.9$，$\beta_2=0.999$，$\epsilon = 10e-8$。

______
#### 如何选择优化算法
1. 如果数据是稀疏的，就用自适应方法，如Adagrad、Adadelta、RMSprop、Adam。
2. Adam就是在RMSprop的基础上加了bias-correction和momentum。
3. 随着梯度变的稀疏，Adam比RMSprop效果会好。

#### 参考文献
1. https://www.cnblogs.com/guoyaohua/p/8542554.html